In [58]:
import spacy
import requests
from spacy import displacy
from IPython import display
from IPython.core.display import display, HTML


In [2]:
with open('pg345.txt') as f:
    dracula = f.read()

In [4]:
!spacy download 'en'

You are using pip version 10.0.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/archy/anaconda/envs/translation/lib/python3.5/site-packages/en_core_web_sm
-->
/Users/archy/anaconda/envs/translation/lib/python3.5/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [10]:
!python -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.1.3) ======================
ℹ spaCy installation:
/Users/archy/anaconda/envs/translation/lib/python3.5/site-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.1.0   ✔
link      en               en_core_web_sm   2.1.0   ✔



In [15]:
nlp = spacy.load('/Users/archy/anaconda/envs/translation/lib/python3.5/site-packages/en_core_web_sm/en_core_web_sm-2.1.0')

In [16]:
snipped = dracula.split('3 May. Bistritz')[-1][0:700]

In [17]:
snipped

'._--Left Munich at 8:35 P. M., on 1st May, arriving at\nVienna early next morning; should have arrived at 6:46, but train was an\nhour late. Buda-Pesth seems a wonderful place, from the glimpse which I\ngot of it from the train and the little I could walk through the\nstreets. I feared to go very far from the station, as we had arrived\nlate and would start as near the correct time as possible. The\nimpression I had was that we were leaving the West and entering the\nEast; the most western of splendid bridges over the Danube, which is\nhere of noble width and depth, took us among the traditions of Turkish\nrule.\n\nWe left in pretty good time, and came after nightfall to Klausenburgh.\nHere I stopped fo'

In [18]:
doc = nlp(snipped)

In [19]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

._--Left ._--Left PUNCT NFP ROOT ._--Xxxx False False
Munich Munich PROPN NNP nsubj Xxxxx True False
at at ADP IN prep xx True True
8:35 8:35 NUM CD nummod d:dd False False
P. P. PROPN NNP compound X. False False
M. M. PROPN NNP pobj X. False False
, , PUNCT , punct , False False
on on ADP IN prep xx True True
1st 1st ADJ JJ amod dxx False False
May May PROPN NNP pobj Xxx True True
, , PUNCT , punct , False False
arriving arrive VERB VBG acl xxxx True False
at at ADP IN prep xx True True

 
 SPACE _SP  
 False False
Vienna Vienna PROPN NNP pobj Xxxxx True False
early early ADV RB advmod xxxx True False
next next ADJ JJ amod xxxx True True
morning morning NOUN NN npadvmod xxxx True False
; ; PUNCT : punct ; False False
should should VERB MD aux xxxx True True
have have VERB VB aux xxxx True True
arrived arrive VERB VBN ROOT xxxx True False
at at ADP IN prep xx True True
6:46 6:46 NUM CD pobj d:dd False False
, , PUNCT , punct , False False
but but CCONJ CC cc xxx True True
train train N

In [20]:
api_key = 'trnsl.1.1.20190310T053025Z.e5636717076895da.44308a26b32209bc0ceaafd5d7c166a661e877f6'

In [21]:
r = requests.post('https://translate.yandex.net/api/v1.5/tr.json/translate',
                 data={'key':api_key,
                 'text':'hello',
                 'lang':'en-fr'})

In [22]:
r.json()

{'code': 200, 'lang': 'en-fr', 'text': ['bonjour']}

In [23]:
def get_translation(word):
    r = requests.post('https://translate.yandex.net/api/v1.5/tr.json/translate',
                 data={'key':api_key,
                 'text':word,
                 'lang':'en-fr'})
    
    return ' '.join(r.json()['text'])
    

## Just translate nouns

In [53]:
output = ''
for token in doc:
    if token.pos_ == 'NOUN' or token.pos_ == 'DET' or token.pos_ == 'ADJ':
        output += get_translation(token.text_with_ws)
    else:
        output += token.text_with_ws

KeyboardInterrupt: 

In [42]:
from IPython.display import IFrame

In [44]:
render_output_to_html(output, 'test.html')

'test.html'

In [45]:
display_html('test.html', width=800)

Just translating nouns is ok, but jarring, because the determinants are missing e.g. 'one heure'

You can't translate the determinants on their own either, going to need to do them together

In [48]:
def mark_translation(text, marker = 'em'):
    
    return '<' + marker + '>' + text + '</' + marker + '>'

In [49]:
trace

['1st ', 'next ', 'morning']

The strategy we adopt here is to start accumulating a "trace" whenever we hit a determinant or an adjective. When we get to the noun, we translate. This seems reasonable because in English adjectives precede nouns.

In [69]:
output = ''
trace = []
for token in doc:
    if token.pos_ == 'DET' or token.pos_ == 'ADJ':
        trace.append(token.text_with_ws)
        print(token.text_with_ws)
    elif token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
        if len(trace) > 0:
            trace.append(token.text_with_ws)
            print('translating', trace)
            output += mark_translation(get_translation(' '.join(trace)))
            trace = []
            print('___')
        else:
            output += mark_translation(get_translation(token.text_with_ws))
    
        
    else:
        if token.pos_ == 'PUNCT' and len(trace) > 0:
            output += mark_translation(get_translation(' '.join(trace)))
            print('translating', trace)
            trace = []


        output += token.text_with_ws

1st 
translating ['1st ', 'May']
___
next 
translating ['next ', 'morning']
___
an
translating ['an', 'hour ']
___
a 
wonderful 
translating ['a ', 'wonderful ', 'place']
___
the 
translating ['the ', 'glimpse ']
___
which 
the 
translating ['which ', 'the ', 'train ']
___
the 
little 
the
translating ['the ', 'little ', 'the', 'streets']
___
the 
translating ['the ', 'station']
___
the 
correct 
translating ['the ', 'correct ', 'time ']
___
possible
translating ['possible']
The
translating ['The', 'impression ']
___
the 
translating ['the ', 'West ']
___
the
translating ['the', 'East']
___
the 
western 
splendid 
translating ['the ', 'western ', 'splendid ', 'bridges ']
___
the 
translating ['the ', 'Danube']
___
which 
noble 
translating ['which ', 'noble ', 'width ']
___
the 
translating ['the ', 'traditions ']
___
Turkish
translating ['Turkish', 'rule']
___
good 
translating ['good ', 'time']
___


In [70]:
display(HTML(output))

Not bad at all. Let's see how we fair with tooltips displaying the original.

In [99]:
def render_output_to_html(html_doc, filename):
    
    full_doc = html_doc['header'] + html_doc['body'] + html_doc['footer']
    
    with open(filename, 'w') as f:
        f.write(full_doc)
        
    return filename

def display_html(filename, width=200, height=200):
    return IFrame(src=filename, width=width, height=height)



In [74]:
display_html('test.html', width=1000)

In [87]:
def add_tooltip_ref(text):
    return f'<div class="htmltooltip"> {text} </div>'

def mark_translation_tooltip(text):
    return f'<a href="#" rel="htmltooltip"> {text} </a>'

In [88]:
html_doc = {'header': None, 
           'body': [],
           'footer': []}

In [127]:
with open('./bbc/sport/001.txt') as f:
    bbc = f.read()

In [129]:
doc2 = nlp(bbc)

In [165]:
with open('hod-raw.txt') as f:
    hod = f.read()

In [166]:
hod_doc = nlp(hod)

In [167]:
len(hod_doc)

49696

In [169]:
output = []
footer = []
trace = []
for i, token in enumerate(hod_doc):
    if token.pos_ == 'DET' or token.pos_ == 'ADJ':
        trace.append(token.text_with_ws)
#         print(token.text_with_ws)
    elif token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
        if len(trace) > 0:
            trace.append(token.text_with_ws)
#             print('translating', trace)
            
            text_ = ' '.join(trace)
            output += [mark_translation_tooltip(get_translation(text_))]
            footer.append(add_tooltip_ref(text_))
            
            trace = []
#             print('___')
        else:
            output += [mark_translation_tooltip(get_translation(token.text_with_ws))]
            footer.append(add_tooltip_ref(token.text_with_ws))
    
    elif len(trace) > 0:
        # If we're still inside a nounphrase, keep accumulating
        trace.append(token.text_with_ws)
    else:
        if token.pos_ == 'PUNCT' and len(trace) > 0:
            # Terminate traces at punctuation, as these correspond to clause endings
            text_ = ' '.join(trace)
            output += [mark_translation_tooltip(get_translation(text_))]
            footer.append(add_tooltip_ref(text_))
#             print('translating', trace)
            trace = []
        else:
            output += [token.text_with_ws]
            
    if i % 1000 == 0:
        print(f"{i} / {len(hod_doc)}")
    

0 / 49696
1000 / 49696
2000 / 49696
3000 / 49696
4000 / 49696
5000 / 49696
6000 / 49696
7000 / 49696
8000 / 49696
9000 / 49696
10000 / 49696


ConnectionError: ('Connection aborted.', OSError(65, 'No route to host'))

In [156]:
def wrap_in_container(text):
    return f"<div class='container'> {text} </div>"

In [157]:
with open('header.html') as f: 
    html_doc['header'] = f.read()

In [170]:
html_doc['body'] = wrap_in_container(''.join(output))

In [171]:
html_doc['footer'] = '\n'.join(footer)

In [177]:
render_output_to_html(html_doc, 'test_doc.html')

'test_doc.html'

In [173]:
display_html('test_doc.html', width = 600)

In [176]:
html_doc['body'] = html_doc['body'].replace('\n', '<br>')